# Preprocessing


In [16]:
!pip install tweet-preprocessor==0.6.0

In [17]:
import pandas as pd
import pickle
import preprocessor as p
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import gensim
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import torch
import torch.nn as nn

In [18]:
data_path = '/all_DEFR_comments_27062022.csv'

# , dtype={'ID':float, 'kommentar_original':str, 'label':int}
df = pd.read_csv(data_path)

<ipython-input-18-2041c2bf9a1a>:4: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [19]:
print(df.shape)

(422046, 16)


In [20]:
df['label'].value_counts()

0    322970
1     99076
Name: label, dtype: int64

In [21]:
import re
import nltk
import string
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download("punkt")
nltk.download('wordnet')
nltk.download('omw-1.4')

def preprocess(data):
    # Remove anything often
    # data.drop(data[data['Hashtags'].str.contains('ALDUB', na=False)].index, inplace=True)

    p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.NUMBER)
    data['text_clean'] = data['kommentar'].apply(lambda x: p.clean(str(x).lower()))

    # Stopword removal, stemming, lemmatisation
    stop_words = set(stopwords.words('german'))
    lemmatizer = WordNetLemmatizer()
    X = data['text_clean'].to_numpy().copy()
    i = 0
    for doc in X:
        sentence = []
        for word in word_tokenize(doc): # word-tokenize sentence
            if not word in stop_words: # remove stop words
                # FIXME hashtags or names
                if not word.startswith("@"):
                    sentence.append(lemmatizer.lemmatize(word)) # lemmatize words
        X[i] = ' '.join(sentence)
        i += 1
    data['text_clean'] = X

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [22]:
def generate_ngrams (tweets, n_gram=1, filter=True):
    stop = stopwords.words("german")
    if filter:
        tokens = [token for tweet in tweets for token in str(tweet).lower().split(" ") if token!="" if token.isalnum() if token not in stop]
    else:
        tokens = [token for tweet in tweets for token in str(tweet).split(" ") if token!=""]
    z = zip(*[tokens[i:] for i in range(n_gram)])
    ngrams = ["_".join(n_gram) for n_gram in z]

    return ngrams

In [23]:
# before preprocessing
print("Generating unigrams and bigrams for raw data ...")
unigrams = generate_ngrams(df['kommentar'], filter=False)
bigrams = generate_ngrams(df['kommentar'], n_gram=2, filter=False)
print("Done.\n")

# preprocessing
print("Preprocessing ...")
preprocess(df)
print("Done.\n")

# after preprocessing
print("Generating unigrams and bigrams for preprocessed data ...")
unigrams_clean = generate_ngrams(df['text_clean'])
bigrams_clean = generate_ngrams(df['text_clean'], n_gram=2)
print("Done.")

Generating unigrams and bigrams for raw data ...
Done.

Preprocessing ...
Done.

Generating unigrams and bigrams for preprocessed data ...
Done.


#### Q4: Dataset splitting (1 pt)

Split your dataset into a training, validation and test set. Motivate your approach for this,
comment on possible evaluation challenges (e.g. label shift over time).
Remark: All subsequent performance scores should be evaluated on your test set.


In [24]:
# Split the dataset over time!
# Sort the data by timestamp, use last 10% for test, 10% of rest for validation.
from sklearn.feature_extraction.text import CountVectorizer

#train, test = train_test_split(df, test_size=0.1, random_state=42, shuffle=False)
df_train = df[:-1000]
test = df[-1000:]
train, val = train_test_split(df_train, test_size=0.2, random_state=42, shuffle=True)

train_X, train_y = train["text_clean"], train["label"]
test_X, test_y = test["text_clean"], test["label"]
val_X, val_y = val["text_clean"], val["label"]

# import pickle
# train_val_test = {'train':(X_train, y_train), 'val':(X_val, y_val), 'test':(X_test,y_test)}
# pickle.dump(train_val_test, open('data/data_splits', 'wb'))

In [25]:
train_y

67875     1
259358    0
275655    0
169955    0
339149    0
         ..
259178    1
365838    0
131932    0
146867    0
121958    0
Name: label, Length: 336836, dtype: int64

#### Classifier


In [26]:
!mkdir -p data
!mkdir -p models

## Transformers BERT


In [27]:
!pip3 install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.1 MB/s eta 0:00:00


In [28]:
# Load model directly
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("statworx/bert-base-german-cased-finetuned-swiss")
model = AutoModelForSequenceClassification.from_pretrained("statworx/bert-base-german-cased-finetuned-swiss", num_labels=1)

text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at statworx/bert-base-german-cased-finetuned-swiss and are newly initialized: ['classifier.weight', 'bert.pooler.dense.weight', 'classifier.bias', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 14.7 MB/s eta 0:00:00


In [30]:
from torch.utils.data import Dataset, DataLoader

class MyBatchedDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data.iloc[index]
        y = torch.tensor(self.labels.iloc[index], dtype = torch.float32)
        return x, y



train_dataset = MyBatchedDataset(train_X, train_y)
val_dataset = MyBatchedDataset(val_X, val_y)
test_dataset = MyBatchedDataset(test_X, test_y)
batch_size = 16
val_batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=val_batch_size, shuffle=False)

In [ ]:
# Batched
import math
from torchmetrics.classification import BinaryF1Score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
torch.manual_seed(0)


#tokenizer = AutoTokenizer.from_pretrained("statworx/bert-base-german-cased-finetuned-swiss")
#model = AutoModelForSequenceClassification.from_pretrained("statworx/bert-base-german-cased-finetuned-swiss", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-german")
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-german", num_labels = 2)
# tokenizer = AutoTokenizer.from_pretrained("deepset/bert-base-german-cased-hatespeech-GermEval18Coarse")
# model = AutoModelForSequenceClassification.from_pretrained("deepset/bert-base-german-cased-hatespeech-GermEval18Coarse", num_labels=2)

#tokenizer = AutoTokenizer.from_pretrained("german-nlp-group/electra-base-german-uncased")
#model = AutoModelForSequenceClassification.from_pretrained("german-nlp-group/electra-base-german-uncased")

#NUM_CLASSES = 2
#model.classifier = torch.nn.Linear(in_features=model.classifier.in_features, out_features=NUM_CLASSES)

#criterion = torch.nn.BCEWithLogitsLoss()
criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=1e-5)


# Freeze specific sublayers
freeze_sublayers = ['encoder.layer.0.', 'encoder.layer.1.', 'encoder.layer.2.',
                    'encoder.layer.3.', 'encoder.layer.4.', 'encoder.layer.5.',
                    'encoder.layer.6.', 'encoder.layer.7.', 'encoder.layer.8.',
                    'encoder.layer.9.', 'encoder.layer.10.']
for name, param in model.named_parameters():
    for freeze_layer in freeze_sublayers:
        if freeze_layer in name:
            param.requires_grad = False

dif_learning_rate = [3e-4, 7e-5]
#optimizer = torch.optim.SGD([{'params': model.bert.encoder.parameters(), 'lr': dif_learning_rate[1]},
#                                             {'params': model.classifier.parameters(), 'lr': dif_learning_rate[0]}], momentum=0.9)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)


model.train()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

loss = None
epochs = 50
best_val_loss = math.inf
iter_no_change = 0
for _ in range(epochs):
    losses = 0
    model.train()
    for idx, (batch_X, batch_Y) in enumerate(train_loader):
        batch_X = tokenizer.batch_encode_plus(batch_X, padding=True, truncation=True, return_tensors='pt')['input_ids']

        batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)

        optimizer.zero_grad()
        try:
          if batch_X.shape[1] > 512:
              batch_X = batch_X[:, 0:512]
          outputs = model(batch_X)
        except:
          print(batch_X.shape[1])


        logits = outputs.logits.squeeze()
        batch_Y = torch.squeeze(batch_Y).long()
        loss = criterion(logits, batch_Y)

        loss.backward()
        optimizer.step()

        losses += loss.item()

    print(f"Train loss after {idx: 04d} / {epochs: 04d} epochs: {losses/len(train_loader): 0.4f}")

    with torch.no_grad():
        model.eval()
        val_loss = 0
        targets, outs = [], []
        for valid_X, valid_Y in val_loader:
            valid_X = tokenizer.batch_encode_plus(valid_X, padding=True, truncation=True, return_tensors='pt')['input_ids']
            valid_X, valid_Y = valid_X.to(device), valid_Y.to(device)

            if valid_X.shape[1] > 512:
                valid_X = valid_X[:, 0:512]

            outputs = model(valid_X)
            logits = outputs.logits
            logits = torch.squeeze(logits)
            #logits = torch.argmax(logits, dim=1)
            #print(logits)


            #outs.extend(logits > 0)
            outs.extend([logits > 0])
            #logits = torch.squeeze(logits)
            targets.extend(valid_Y)
            valid_Y = valid_Y.unsqueeze(0)
            valid_Y = valid_Y[0, :]
            valid_Y = torch.squeeze(valid_Y).long()
            val_loss += criterion(logits, valid_Y).item()
        val_loss = val_loss / len(val_loader)
        scheduler.step(val_loss)
        print(f"  Val loss after {idx: 04d} / {epochs: 04d} epochs: {val_loss: 0.4f}")

        f1 = BinaryF1Score()
        #outs_t, targets_t = torch.cat(outs, dim=0).long(), torch.cat(targets, dim=0).long()

        #print(targets_t.shape)
        #print(outs_t.shape)

        #print(f"  Val F1 after {idx: 04d} / {epochs: 04d} epochs: {f1(outs_t, targets_t): 0.4f}")

        if val_loss < best_val_loss:
            torch.save(model, f'models/swiss_bert_model_{val_loss}.pt')
            print("model saved!")
#                     torch.save(classifier, 'ger_classifier.pt')
            best_val_loss = val_loss
            iter_no_change = 0
        if val_loss >= best_val_loss:
            iter_no_change += 1
            #if iter_no_change > 10:
                #print(f"Early stopping: val loss didn't improve for {10*100*idx} steps!")
                #break
    #if iter_no_change > 10:
        #break

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
import numpy as np

name_model = '/content/dirty/swiss_bert_model_0.5350865106517022.pt'

test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)
model = torch.load(name_model)
model.eval()

with torch.no_grad():
    y_pred = []
    test_loss = 0
    for test_X, test_Y in test_loader:
        test_X = tokenizer.batch_encode_plus(test_X, padding=True, truncation=True, return_tensors='pt')['input_ids']
        test_X, test_Y = test_X.to(device), test_Y.to(device)
        outputs = model(test_X)
        logits = outputs.logits
        #y_pred = y_pred.tolist()
        y_pred.append(np.argmax(logits.cpu().numpy(), axis=1))
        #y_pred = np.argmax(y_pred, axis=1)
    y_pred = np.concatenate(y_pred)
    y_test = test_dataset.labels
    mae = mean_absolute_error(y_test, y_pred)

    score_f1 = f1_score(np.round(y_test).astype(int), np.round(y_pred).astype(int), average='weighted')

    print("f1:", score_f1)

In [ ]:
print(np.mean(y_pred == y_test.values))

In [ ]:
class TestDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data.iloc[index]
        return x


In [ ]:
test = pd.read_csv('test_without_labels.csv', dtype={'text':str})


In [ ]:
# Dev
# eval_dev_data = pd.read_csv('dev.csv', dtype={'text':str, 'label':int})

# FIXME
# preprocess(test)

test_X = test["text"]
test_dataset = TestDataset(test_X)
batch_size = 1

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# model = torch.load('/content/swiss_bert_model_0.5350865106517022.pt') #0.72
model = torch.load('/content/dirty/swiss_bert_model_0.5350865106517022.pt')
model.eval()


with torch.no_grad():
    y_pred = []
    test_loss = 0
    for test_X in test_loader:
        test_X = tokenizer.batch_encode_plus(test_X, padding=True, truncation=True, return_tensors='pt')['input_ids']
        test_X = test_X.to(device)
        outputs = model(test_X)
        logits = outputs.logits
        y_pred.append(np.argmax(logits.cpu().numpy(), axis=1))
    y_pred = np.concatenate(y_pred)

    # # Dev
    # y_test = eval_dev_data['label']
    # mae = mean_absolute_error(y_test, y_pred)
    # score_f1 = f1_score(np.round(y_test).astype(int), np.round(y_pred).astype(int),average='weighted')

    # print("mae:", mae)
    # print("f1:", score_f1)

    test["label"] = y_pred

test.to_csv('test_pred3.csv', index=False)